# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [1]:
from nltk.lm.models import KneserNeyInterpolated
from nltk.util import ngrams
from nltk.lm.preprocessing import padded_everygram_pipeline, pad_both_ends

import pandas as pd
from tqdm import tqdm

# prepare data
df = pd.read_csv("fivegrams.txt", sep="\t", header=None, names=['frequency', 'word1', 'word2', 'word3', 'word4', 'word5'])
sequences = []
frequencies = []
for _, row in tqdm(df.iterrows()):
    frequency = row['frequency']
    ngram = [row['word1'], row['word2'], row['word3'], row['word4'], row['word5']]
    sequences.extend([ngram] * 1)
    frequencies.extend([frequency] * 1)    


1044268it [01:30, 11497.48it/s]


In [9]:
order = 5
# Manually add padding tokens at the beginning of each sequence
sequences_padded = [['<s>'] * (order - 1 ) + seq for seq in sequences[:100000]]
train_data_padded, vocab = padded_everygram_pipeline(order, sequences_padded)

# Initialize and train the Kneser-Ney interpolated model
kn_model = KneserNeyInterpolated(order)
kn_model.fit(train_data_padded, vocab)

# Example usage: scoring a word given a context
context = ('a', 'baby', 'in','the')
word = 'indigestion'
score = kn_model.score(word, context)
print(f"Score of '{word}' given context '{context}': {score}")

Score of 'indigestion' given context '('a', 'baby', 'in', 'the')': 1.0418361103558039e-10


In [3]:
context = ('a', 'kid', 'in','new')
word = 'baby'
score = kn_model.score(word, context)
print(f"Score of '{word}' given context '{context}': {score}")

Score of 'baby' given context '('a', 'baby', 'in', 'new')': 1.0507223787618395e-05


In [4]:
context = ('a', 'baby', 'in','new')
word = 'york'
score = kn_model.score(word, context)
print(f"Score of '{word}' given context '{context}': {score}")

Score of 'york' given context '('a', 'baby', 'in', 'new')': 0.7077801584613578


In [5]:
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return list(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    edits1_result = edits1(word)
    edits2_result = (edits1(e1) for e1 in edits1_result)
    
    edits2_in_vocab = (edit for edit in edits2_result if edit in vocab)
    
    return list(edits2_in_vocab)
def candidates(word): 
    "Generate possible spelling corrections for word."
    return edits2(word) + edits1(word) + [word]



def P(word, context=('<s>', '<s>', '<s>', '<s>')):  
    adjusted_context = context[-(order-1):] if len(context) >= order-1 else ('<s>',)*(order-1-len(context)) + tuple(context)
    # Calculate and return the score of the word given its context
    return kn_model.score(word, adjusted_context)

def correction(word, context=('<s>', '<s>', '<s>', '<s>')): 
    
    # Generate possible spelling corrections for the word
    return max(candidates(word), key=lambda w: P(w, context))


In [6]:
correction("horse", context=('a', 'baby', 'in','his'))

'house'

In [7]:
correction("tima", context=('student', 'does', "n't",'have'))

'time'

## Justify your decisions

### Kneser-Ney Interpolated Model

Effectively addresses the challenge of sparse data, which is common in language modeling, by distributing probability mass to unseen n-grams in a nuanced manner.

Context Sensitivity: Leverages the context provided by preceding words to predict the likelihood of the next word, crucial for tasks requiring understanding of word usage in context, such as spell correction.

Robustness and Generalization: Offers better generalization to new texts by dealing effectively with zero-frequency problems, making the model robust across diverse inputs.

Interpolation for Enhanced Estimates: Uses interpolation to combine probabilities from different n-gram levels, utilizing both specific and broader context for more accurate probability estimates.

Empirical Success: Has been empirically shown to perform well across various datasets and tasks, establishing it as a preferred choice for language modeling.

### Choice of 5-gram (Quint-grams)
Increased Context Sensitivity: Provides a richer contextual basis for predictions by considering the four preceding words, improving the model’s ability to capture linguistic nuances.

Balance Between Specificity and Generalization: Offers a middle ground that captures sufficient context for usefulness while avoiding overfitting, thus maintaining the model’s ability to generalize from training data.

Computational Feasibility: Represents a practical upper limit for n-gram size, balancing the benefits of additional context against the computational and memory requirements.
 
 Diminishing Returns Beyond 5-grams: Acknowledges that while higher n-grams could provide more context, the performance improvement may not justify the increased computational complexity.


These points illustrate the thoughtful considerations behind selecting the Kneser-Ney Interpolated model and a 5-gram approach, aiming to optimize for context sensitivity, computational efficiency, and model effectiveness in handling the nuances of natural language.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [8]:
import pandas as pd
import random
def generate_misspelled_word(word):
    possible_edits = list(edits1(word)) + list(edits2(word))
    if not possible_edits:
        return word
    return random.choice(possible_edits)
def report_accuracy(df):
    total_words = 0
    successful_corrections = 0

    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Split sentence into words
        words = row['Sentence'].lower().split()
        
        # Iterate over each word in the sentence
        for word in words:
            total_words += 1
            context = ["<s>"]
            # Generate misspelled word
            misspelled_word = generate_misspelled_word(word)
            
            # Correct misspelled word
            corrected_word = correction(misspelled_word, context)
            context.append(corrected_word)
            # Check if corrected word matches original word
            if corrected_word == word:
                successful_corrections += 1

    # Calculate accuracy
    accuracy = successful_corrections / total_words if total_words > 0 else 0
    return accuracy

df = pd.read_csv("custom_data.csv", encoding = "latin-1")
report_accuracy(df)

0.8571428571428571